In [1]:
import sys
sys.path.append('./src')
sys.path.append('./dependencies')

from utils import *
from sqp import *
from osqp_problem import *
from osqp_mpc import OsqpMpc
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import meshcat.geometry as g
import meshcat.transformations as tf
import numpy as np
np.set_printoptions(linewidth=99999999)


In [2]:

config = {
    'N': 32,
    'dt': 0.01,
    'joint_vel_cost': 1e-3,
    'control_cost': 1e-5,
    'terminal_cost': 100, # not used
    'mu': 10.0,
    'sim_dt': 0.001
}

num_steps = 500

urdf_filename = "dependencies/indy-ros2/indy_description/urdf_files/indy7.urdf"

mpc = OsqpMpc(urdf_filename, config)

xstart = np.hstack((np.ones(6), np.zeros(6)))

endpoints = np.array([
    np.array(solver.eepos(np.zeros(6))), 
#    np.array(solver.eepos(-0.8 * np.ones(nq)))
])
print("Endpoints: {}".format(endpoints))


In [ ]:
viz = MeshcatVisualizer(mpc.robot.model, mpc.robot.collision_model, mpc.robot.visual_model)
viz.initViewer()
viz.loadViewerModel()

# Visualize goals
for i, p in enumerate(endpoints):
    viz.viewer[f'point{i}'].set_object(
        g.Sphere(0.05), 
        g.MeshLambertMaterial(color=0xff0000))

    T = tf.translation_matrix(np.array(p[:]))
    viz.viewer[f'point{i}'].set_transform(T)

In [20]:
# Run 
q_trajectory = mpc.run_mpc(xstart, endpoints, num_steps)

In [21]:
print_stats(sqp.get_stats())
viz.play(q_trajectory=q_trajectory, dt=0.01)